# Create the regionalized version of EF 3.1 in brightway

This notebook should be run twice: 
- a first time to create the updated spatialized biosphere database with `name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'`, `write_method_bw = False` and `write_new_spatialized_biosphere = True`. This will create the spatialized biosphere database in the brightway project, as well as the associated pickle file.
- and a second time to create the regionalized methods with `name_spatialized_biosphere_db = 'biosphere3_spatialized_flows_EF'`, `write_method_bw = True` and `write_new_spatialized_biosphere = False`. This will create the EF 3.1 regionalized method in the brightway project, as well as the associated bw2package file. This second iteration requires the adjusted spatialized biosphere database to be present in the brightway project.

In [1]:
import bw2data as bd
import bw2io as bi
import numpy as np
import pandas as pd
from mescal import *
import os

## User inputs

In [2]:
bd.projects.set_current('ecoinvent3.9.1A')

In [3]:
name_spatialized_biosphere_db = 'biosphere3_spatialized_flows_EF'

In [4]:
write_method_bw = True
write_new_spatialized_biosphere = False

## Set up

In [5]:
cwd = os.getcwd()

In [6]:
ef_cfs = pd.read_excel('data/EF-LCIAMethod_CF(EF-v3.1).xlsx', sheet_name='lciamethods_CF', keep_default_na=False, na_values='')
ef_biosphere_mapping = pd.read_csv('data/EF31_biosphere_mapping.csv')

In [7]:
ef_biosphere_mapping_dict = ef_biosphere_mapping.groupby('ef')['biosphere'].apply(list).to_dict()

## Adapt the EF excel file to be compatible with ecoinvent

In [8]:
def ecoinvent_compatible_categories(name, cat0, cat1, cat2):
    
    if name.lower() in ef_biosphere_mapping_dict.keys():
        name = ef_biosphere_mapping_dict[name.lower()][0]
    
    if cat0 == 'Emissions':
        cat1 = cat1.replace('Emissions to ', '')
        cat2 = cat2.replace('Emissions to ', '')
        cat2 = cat2.replace('and', '+')
        
        if cat2 == 'air, unspecified':
            cat2 = None
        elif cat2 == 'air, unspecified (long-term)':
            cat2 = 'low population density, long-term'
        elif cat2 == 'non-agricultural soil':
            cat2 = 'industrial'
            
        if '(biogenic)' in name:
            name = name.replace(' (biogenic)', ', non-fossil')
        elif '(fossil)' in name:
            name = name.replace(' (fossil)', ', fossil')
        elif '(land use change)' in name:
            name = name.replace(' (land use change)', ', to soil or biomass stock')
        
        if 'Particles' in name:
            name.replace('Particles', 'Particulate Matter, ')
            
            if '(PM10)' in name:
                name = name.replace('(PM10)', '> 2.5 um and < 10um')
            elif '(PM0.2 - PM2.5)' in name:
                name = name.replace('(PM0.2 - PM2.5)', '< 2.5 um')
    
    elif cat0 == 'Land use':
        
        if ('from ' in name) & (name.replace('from ', '') in ef_biosphere_mapping_dict.keys()):
            name = f"from {ef_biosphere_mapping_dict[name.replace('from ', '')][0]}"
        
        elif ('to ' in name) & (name.replace('to ', '') in ef_biosphere_mapping_dict.keys()):
            name = f"to {ef_biosphere_mapping_dict[name.replace('to ', '')][0]}"
        
        cat1 = cat1.replace('Land ', '')
        name = f'{cat1}, {name}'
        cat1 = 'natural resource'
        cat2 = 'land'
    
    elif cat0 == 'Resources':
        cat2 = f"in {cat1.replace('Resources from ', '')}"
        cat1 = 'natural resource'
    
    else:
        raise ValueError(f"Category0 {cat0} not recognized")
    
    return name, cat0, cat1, cat2

In [9]:
def ecoinvent_compatible_water_flows(name, cat0, cat1, cat2, lcia_cat):
    
    name = name.lower()
    
    if lcia_cat.lower() == 'water use':
        
        if cat0 == 'Resources':
            if name == 'ground water':
                name = 'water, well, in ground'
            elif name == 'lake water':
                name = 'water, lake'
            elif name == 'river water':
                name = 'water, river'
            elif name == 'freshwater':
                name = 'water, unspecified natural origin'
            elif name == 'water to cooling':
                name = 'water, cooling, unspecified natural origin'
            elif name == 'water to turbine':
                name = 'water, turbine use, unspecified natural origin'
            
            cat1 = 'natural resource'
            cat2 = None
        
        elif cat0 == 'Emissions':
            if (name == 'water') & (cat2 == 'water, unspecified'):
                name = 'water'
                cat1 = 'water'
                cat2 = None
        
    return name, cat1, cat2

In [10]:
# Change the name and categories columns of the EF file to be ecoinvent-compliant 
ef_cfs[['FLOW_name', 'FLOW_class0', 'FLOW_class1', 'FLOW_class2']] = ef_cfs.apply(lambda x: ecoinvent_compatible_categories(x['FLOW_name'], x['FLOW_class0'], x['FLOW_class1'], x['FLOW_class2']), axis=1, result_type='expand')

In [11]:
ef_cfs[['FLOW_name', 'FLOW_class1', 'FLOW_class2']] = ef_cfs.apply(lambda x: ecoinvent_compatible_water_flows(x['FLOW_name'], x['FLOW_class0'], x['FLOW_class1'], x['FLOW_class2'], x['LCIAMethod_name']), axis=1, result_type='expand')

In [12]:
# Add some missing land biosphere flows, as they come from the same EF flow
ef_cfs.drop_duplicates(inplace=True)
for key, value in ef_biosphere_mapping_dict.items():
    if len(value) > 1:
        df_to_add = ef_cfs[
            (ef_cfs.FLOW_name.str.lower() == f'occupation, {value[0]}')
            | (ef_cfs.FLOW_name.str.lower() == f'transformation, from {value[0]}')
            | (ef_cfs.FLOW_name.str.lower() == f'transformation, to {value[0]}')
        ].copy(deep=True)
        for i in range(1, len(value)):
            df_to_add['FLOW_name'] = df_to_add['FLOW_name'].apply(lambda x: x.replace(value[i-1], value[i]))
            ef_cfs = pd.concat([ef_cfs, df_to_add])

ef_cfs.reset_index(drop=True, inplace=True)

In [13]:
# Add unspecified water flow 
df_to_add = ef_cfs[
    (ef_cfs.FLOW_name.str.lower() == 'water, unspecified natural origin')
    & (ef_cfs.FLOW_class1 == 'natural resource')
    & (ef_cfs.FLOW_class2.isna())
].copy(deep=True)
df_to_add['FLOW_name'] = 'water'
ef_cfs = pd.concat([ef_cfs, df_to_add])

# If needed, add missing water flows (copies from water flow without second category)
missing_subcategories_resources = ['fossil well', 'in ground', 'in water']
water_resource_flows = ['water', 'water, lake', 'water, river', 'water, unspecified natural origin', 'water, cooling, unspecified natural origin', 'water, turbine use, unspecified natural origin', 'water, well, in ground']
for water_flow in water_resource_flows:
    df_to_add = ef_cfs[
        (ef_cfs.FLOW_name.str.lower() == water_flow)
        & (ef_cfs.FLOW_class1 == 'natural resource')
        & (ef_cfs.FLOW_class2.isna())
    ].copy(deep=True)
    for subcat in missing_subcategories_resources:
        df_to_add['FLOW_class2'] = subcat
        ef_cfs = pd.concat([ef_cfs, df_to_add])

missing_subcategories_emissions = ['fossil well', 'ground-', 'ground-, long-term', 'surface water']
df_to_add = ef_cfs[
    (ef_cfs.FLOW_name.str.lower() == 'water')
    & (ef_cfs.FLOW_class1 == 'water')
    & (ef_cfs.FLOW_class2.isna())
].copy(deep=True)
for subcat in missing_subcategories_emissions:
    df_to_add['FLOW_class2'] = subcat
    ef_cfs = pd.concat([ef_cfs, df_to_add])

ef_cfs.reset_index(drop=True, inplace=True)

In [14]:
# # Add a RoW and a GLO row in addition to the normal one (with no location) 
# df_to_add = ef_cfs[ef_cfs.LCIAMethod_location.isna()].copy(deep=True)
# df_to_add['LCIAMethod_location'] = 'GLO'
# ef_cfs = pd.concat([ef_cfs, df_to_add])
# 
# df_to_add = ef_cfs[ef_cfs.LCIAMethod_location.isna()].copy(deep=True)
# df_to_add['LCIAMethod_location'] = 'RoW'
# ef_cfs = pd.concat([ef_cfs, df_to_add])

In [15]:
# Load the biosphere and spatialized biosphere databases 
biosphere_db_spatialized = Database(name_spatialized_biosphere_db)
biosphere_db = Database('biosphere3')

Loaded biosphere3_spatialized_flows_EF from pickle!
Getting activity data


100%|██████████| 4718/4718 [00:00<00:00, 184634.22it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 4718/4718 [00:00<00:00, 771700.90it/s]

Loaded biosphere3 from brightway!


In [16]:
biosphere_db_dict_name = biosphere_db.list_to_dict('name', 'biosphere')
biosphere_db_spatialized_dict_name = biosphere_db_spatialized.list_to_dict('name', 'biosphere')

In [17]:
biosphere_db_dict_code = biosphere_db.list_to_dict('code', 'biosphere')
biosphere_db_spatialized_dict_code = biosphere_db_spatialized.list_to_dict('code', 'biosphere')

## Create the new regionalized methods

In [18]:
def create_cf_list(df):
    
    reg_cfs = []
    unmapped = []
    
    for i in range(len(df)):
        
        flow_name = df.loc[i, 'FLOW_name'].lower()
        flow_location = df.loc[i, 'LCIAMethod_location']
        flow_cf = df.loc[i, 'CF EF3.1']
        
        if flow_location is np.nan:
            spatialized_db = False
        else:
            flow_name = f'{flow_name}, {flow_location}'
            spatialized_db = True
        
        category1 = df.loc[i, 'FLOW_class1']
        category2 = df.loc[i, 'FLOW_class2']
        
        flow_name = flow_name[0].upper() + flow_name[1:]
        
        if category2 is not None:
            
            if spatialized_db:
                if (flow_name, (category1, category2), name_spatialized_biosphere_db) in biosphere_db_spatialized_dict_name.keys():
                    flow_code = biosphere_db_spatialized_dict_name[(flow_name, (category1, category2), name_spatialized_biosphere_db)]['code']
                    reg_cfs.append(((name_spatialized_biosphere_db, flow_code), flow_cf))
                else:
                    unmapped.append((name_spatialized_biosphere_db, flow_name, (category1, category2)))
            
            else:
                if (flow_name, (category1, category2), 'biosphere3') in biosphere_db_dict_name.keys():
                    flow_code = biosphere_db_dict_name[(flow_name, (category1, category2), 'biosphere3')]['code']
                    reg_cfs.append((('biosphere3', flow_code), flow_cf))
                else:
                    unmapped.append(('biosphere3', flow_name, (category1, category2)))
        
        else:
            
            if spatialized_db:
                if (flow_name, (category1,), name_spatialized_biosphere_db) in biosphere_db_spatialized_dict_name.keys():
                    flow_code = biosphere_db_spatialized_dict_name[(flow_name, (category1,), name_spatialized_biosphere_db)]['code']
                    reg_cfs.append(((name_spatialized_biosphere_db, flow_code), flow_cf))
                else:
                    unmapped.append((name_spatialized_biosphere_db, flow_name, (category1,)))
            
            else:
                if (flow_name, (category1,), 'biosphere3') in biosphere_db_dict_name.keys():
                    flow_code = biosphere_db_dict_name[(flow_name, (category1,), 'biosphere3')]['code']
                    reg_cfs.append((('biosphere3', flow_code), flow_cf))
                else:
                    unmapped.append(('biosphere3', flow_name, (category1,)))
    
    return reg_cfs, unmapped

In [19]:
method_names_bw_to_ef = {
    'acidification': 'Acidification',
    'climate change': 'Climate change',
    'climate change: biogenic': 'Climate change-Biogenic',
    'climate change: fossil': 'Climate change-Fossil',
    'climate change: land use and land use change': 'Climate change-Land use and land use change',
    'particulate matter formation': 'EF-particulate Matter',
    'ecotoxicity: freshwater': 'Ecotoxicity, freshwater',
    'ecotoxicity: freshwater, inorganics': 'Ecotoxicity, freshwater_inorganics',
    'ecotoxicity: freshwater, organics': 'Ecotoxicity, freshwater_organics',
    'eutrophication: marine': 'Eutrophication marine',
    'eutrophication: freshwater': 'Eutrophication, freshwater',
    'eutrophication: terrestrial': 'Eutrophication, terrestrial',
    'human toxicity: carcinogenic': 'Human toxicity, cancer',
    'human toxicity: carcinogenic, inorganics': 'Human toxicity, cancer_inorganics',
    'human toxicity: carcinogenic, organics': 'Human toxicity, cancer_organics',
    'human toxicity: non-carcinogenic': 'Human toxicity, non-cancer',
    'human toxicity: non-carcinogenic, inorganics': 'Human toxicity, non-cancer_inorganics',
    'human toxicity: non-carcinogenic, organics': 'Human toxicity, non-cancer_organics',
    'ionising radiation: human health': 'Ionising radiation, human health',
    'land use': 'Land use',
    'ozone depletion': 'Ozone depletion',
    'photochemical oxidant formation: human health': 'Photochemical ozone formation - human health',
    'energy resources: non-renewable': 'Resource use, fossils',
    'material resources: metals/minerals': 'Resource use, minerals and metals',
    'water scarcity': 'Water use'
}

In [20]:
# Create the new set of regionalized impact categories 
ef_methods_bw = [i for i in bd.methods if i[0] == 'EF v3.1']

# Replace the water use method of EF by the one of IW+. Both are based on AWARE, but the one of IW+ is more complete. 
ef_methods_bw.remove(('EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)'))
ef_methods_bw.append(('IMPACT World+ Midpoint 2.0.1', 'Midpoint', 'Water scarcity'))

regionalized_methods = {}
unlinked = []

# Acidification, terrestrial eutrophication, land use, and water use are the four categories to be fully regionalized (i.e., fo every elementary flow). For the other categories, only a subset of flows is regionalized (the ones in common with the regionalized methods).
methods_to_regionalize = [
    'acidification',
    'eutrophication: terrestrial',
    'eutrophication: marine', # ammonia, nitrogen dioxide, and nitrogen oxides flows only (with global value)
    'ecotoxicity: freshwater', # ammonia flows only (with global value)
    'ecotoxicity: freshwater, inorganics', # ammonia flows only (with global value)
    'human toxicity: non-carcinogenic', # ammonia flows only (with global value)
    'human toxicity: non-carcinogenic, inorganics', # ammonia flows only (with global value)
    'land use',
    'photochemical oxidant formation: human health', # nitrogen dioxide, nitrogen oxides, sulfur dioxide, and sulfur oxides flows only (with global value)
    'water scarcity',
] # categories for which regional CF are added, other categories are kept as they are in brightway

for i in range(len(ef_methods_bw)):
    method = bd.Method(ef_methods_bw[i])
    if ef_methods_bw[i][0] == 'EF v3.1':
        method_name_bw = ef_methods_bw[i][1]
    else:
        method_name_bw = ef_methods_bw[i][2] # for the IW+ water scarcity method
    method_name = method_names_bw_to_ef[method_name_bw.lower()]
    cfs = method.load()
    regionalized_cfs = []
    
    for j in cfs:
        if method_name_bw.lower() not in methods_to_regionalize:
            regionalized_cfs.append(j) # non-regionalized flows are kept as they are
        
        else:
            flow = biosphere_db_dict_code[(j[0][0], j[0][1])]
            flow_name = flow['name'].lower()
            
            # for the partially regionalized methods, only some flows are regionalized
            if method_name_bw.lower() in [
                'ecotoxicity: freshwater', 
                'ecotoxicity: freshwater, inorganics', 
                'human toxicity: non-carcinogenic', 
                'human toxicity: non-carcinogenic, inorganics'
            ]:
                if flow_name != 'ammonia':
                    regionalized_cfs.append(j)
                    continue
            elif method_name_bw.lower() == 'eutrophication: marine':
                if flow_name not in ['ammonia', 'nitrogen dioxide', 'nitrogen oxides']:
                    regionalized_cfs.append(j)
                    continue
            elif method_name_bw.lower() == 'photochemical oxidant formation: human health':
                if flow_name not in ['nitrogen dioxide', 'nitrogen oxides', 'sulfur dioxide', 'sulfur oxides']:
                    regionalized_cfs.append(j)
                    continue
            
            # for the fully regionalized methods, all flows are regionalized
            flow_category_1 = flow['categories'][0]
            
            if len(flow['categories']) == 1:
                df_regionalized_flows = ef_cfs[
                    (ef_cfs['FLOW_name'].str.lower() == flow_name) 
                    & (ef_cfs['FLOW_class1'] == flow_category_1)
                    & (ef_cfs['FLOW_class2'].isna())
                    & (ef_cfs['LCIAMethod_name'] == method_name)
                ].reset_index()
                
                if len(df_regionalized_flows) == 0:
                    unlinked.append(('biosphere3', flow_name, (flow_category_1,)))
                
            else:
                flow_category_2 = flow['categories'][1]
            
                df_regionalized_flows = ef_cfs[
                    (ef_cfs['FLOW_name'].str.lower() == flow_name) 
                    & (ef_cfs['FLOW_class1'] == flow_category_1)
                    & (ef_cfs['FLOW_class2'] == flow_category_2)
                    & (ef_cfs['LCIAMethod_name'] == method_name)
                ].reset_index()
                
                if len(df_regionalized_flows) == 0:
                    unlinked.append(('biosphere3', flow_name, (flow_category_1, flow_category_2)))
            
            new_regionalized_cfs, new_unlinked = create_cf_list(df_regionalized_flows)
            regionalized_cfs += new_regionalized_cfs
            unlinked += new_unlinked
    
    regionalized_methods[method_name_bw] = regionalized_cfs

In [21]:
# Check if some global flows are still unlinked.
list(set([i[1:] for i in unlinked if 'biosphere3' == i[0]]))

[('water', ('water', 'ocean'))]

The water (water, ocean) flow is missing, but this is not a problem as its CF is 0. 

In [22]:
if write_method_bw:
    for method in ef_methods_bw:
        if method[2] == 'Water scarcity':
            ef_regionalized_method = bd.Method(('EF v3.1 regionalized', 'water use', 'user deprivation potential (deprivation-weighted water consumption)'))
            unit = bd.Method(('EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)')).metadata['unit']
            method_name = method[2]
        else:
            ef_regionalized_method = bd.Method(('EF v3.1 regionalized', method[1], method[2]))
            unit = bd.Method(method).metadata['unit']
            method_name = method[1]
        ef_regionalized_method_metadata = {'unit': unit}
        ef_regionalized_method.register(**ef_regionalized_method_metadata)
        ef_regionalized_method.write(regionalized_methods[method_name])

In [23]:
list_ef_regionalized_methods = [bd.Method(('EF v3.1 regionalized', method[1], method[2])) for method in ef_methods_bw if method[2] != 'Water scarcity']
list_ef_regionalized_methods.append(bd.Method(('EF v3.1 regionalized', 'water use', 'user deprivation potential (deprivation-weighted water consumption)')))

In [24]:
if write_method_bw:
    bi.package.BW2Package.export_objs(list_ef_regionalized_methods, filename='EF3.1_regionalized', folder=cwd+'\\results\\')

## Spatialized flows from regioinvent that are not used by EF regionalized methods

There might be some spatialized elementary flows that are in the spatialized biosphere database from regioinvent that are not used in any EF regionalized method. Those flows should be removed. Otherwise, the resulting spatialized flows in the ecoinvent database will not be characterized, thus possibly leading to an underestimation of impacts, as the corresponding global (non-spatialized) flow could be characterized in EF. 

In [25]:
all_regionalized_flows_in_ef = list(set([i[0][1] for i in [x for xs in regionalized_methods.values() for x in xs] if i[0][0] == name_spatialized_biosphere_db]))

In [26]:
all_regionalized_flows_in_regioinvent = [i['code'] for i in biosphere_db_spatialized.db_as_list]

In [27]:
used_spatialized_flows = list(set(all_regionalized_flows_in_regioinvent).intersection(all_regionalized_flows_in_ef))

In [28]:
new_spatialized_biosphere_db = Database(db_as_list=[i for i in biosphere_db_spatialized.db_as_list if i['code'] in used_spatialized_flows])

## Missing locations and flows 

In [29]:
ei_database = Database('ecoinvent-3.9.1-cutoff')

Loaded ecoinvent-3.9.1-cutoff from pickle!


In [30]:
ei_locations = list(set([i['location'] for i in ei_database.db_as_list]))

In [31]:
missing_biosphere_flows = sorted(list(set([i[1].rsplit(', ', 1)[0] for i in unlinked])))

In [32]:
missing_flows_dict = {}
for flow in missing_biosphere_flows:
    missing_loc = list(set([i[1].rsplit(', ', 1)[-1] for i in unlinked if (i[1].rsplit(', ', 1)[0] == flow) & (i[1].rsplit(', ', 1)[-1] in ei_locations)]))
    if len(missing_loc) > 0:
        print(f'##### {flow} #####')
        print(f'{sorted(missing_loc)}\n')
        missing_flows_dict[flow] = missing_loc

##### Occupation, annual crop, flooded crop #####
['AE', 'BH', 'CW', 'CY', 'EG', 'GI', 'IL', 'IS', 'JO', 'KW', 'LB', 'LY', 'MT', 'OM', 'QA', 'SA', 'SY', 'TJ', 'TM', 'UZ', 'YE']

##### Occupation, annual crop, greenhouse #####
['AE', 'BH', 'CW', 'CY', 'EG', 'GI', 'IL', 'IS', 'JO', 'KW', 'LB', 'LY', 'MT', 'OM', 'QA', 'SA', 'SY', 'TJ', 'TM', 'UZ', 'YE']

##### Occupation, annual crop, irrigated #####
['AE', 'BH', 'CW', 'CY', 'EG', 'GI', 'IL', 'IS', 'JO', 'KW', 'LB', 'LY', 'MT', 'OM', 'QA', 'SA', 'SY', 'TJ', 'TM', 'UZ', 'YE']

##### Occupation, annual crop, irrigated, extensive #####
['AE', 'BH', 'CW', 'CY', 'EG', 'GI', 'IL', 'IS', 'JO', 'KW', 'LB', 'LY', 'MT', 'OM', 'QA', 'SA', 'SY', 'TJ', 'TM', 'UZ', 'YE']

##### Occupation, annual crop, irrigated, intensive #####
['AE', 'BH', 'CW', 'CY', 'EG', 'GI', 'IL', 'IS', 'JO', 'KW', 'LB', 'LY', 'MT', 'OM', 'QA', 'SA', 'SY', 'TJ', 'TM', 'UZ', 'YE']

##### Occupation, annual crop, non-irrigated #####
['AE', 'BH', 'CW', 'CY', 'EG', 'GI', 'IL', 'IS',

However, all of these flows are not necessarily used in ecoinvent. 

### Add missing biosphere flows to regioinvent's biosphere database

In [33]:
# Check if those flows would be used in ecoinvent database
missing_flows_used_in_ei = []
all_missing_loc = list(set([i[1].rsplit(', ', 1)[-1] for i in unlinked])) # all missing locations retrieved from the unlinked list
for loc in all_missing_loc:
    missing_flows_for_loc = [i[1].rsplit(', ', 1)[0] for i in unlinked if i[1].rsplit(', ', 1)[-1] == loc] # all types of unlinked flows with this location
    activities_of_loc = [i for i in ei_database.db_as_list if i['location'] == loc] # list of ecoinvent activities for this location
    for act in activities_of_loc:
        biosphere_flows = Dataset(act).get_biosphere_flows() # list of biosphere flows in the activity exchanges 
        for flow in biosphere_flows:
            if flow['name'] in missing_flows_for_loc:
                missing_flows_used_in_ei.append([act["name"], act["location"], flow["name"], flow["categories"], flow["unit"]])

In [34]:
# create a dictionary with the useful missing flows as key (name, categories, unit) and the list of missing locations as values 
missing_flows_used_in_ei_dict = {}
counter = 0
for i in missing_flows_used_in_ei:
    if (i[2], i[3], i[4]) in missing_flows_used_in_ei_dict.keys():
        if i[1] not in missing_flows_used_in_ei_dict[(i[2], i[3], i[4])]:
            missing_flows_used_in_ei_dict[(i[2], i[3], i[4])].append(i[1])
            counter += 1
    else:
        missing_flows_used_in_ei_dict[(i[2], i[3], i[4])] = [i[1]]
        counter += 1

In [35]:
# for each missing flow and associated list of locations, create a new flow in the spatialized biosphere database
for key in missing_flows_used_in_ei_dict.keys():
    for loc in missing_flows_dict[key[0]]:
        new_biosphere_act = {
            'name': f'{key[0]}, {loc}',
            'categories': key[1],
            'unit': key[2],
            'code': random_code(),
            'database':'biosphere3_spatialized_flows_EF',
        }
        new_spatialized_biosphere_db.db_as_list.append(new_biosphere_act)

In [36]:
if write_new_spatialized_biosphere:
    new_spatialized_biosphere_db.write_to_brightway(new_db_name='biosphere3_spatialized_flows_EF', database_type='biosphere')

In [37]:
if write_new_spatialized_biosphere:
    new_spatialized_biosphere_db = Database('biosphere3_spatialized_flows_EF', create_pickle=True)

## Geographical mapping between ecoinvent (including premise regions) and EF methods to be used in regioinvent

In [38]:
ei_premise_iam_locations_geo_mapping = {
    'CAZ': 'CA',
    'CHA': 'CN',
    'EUR': 'RER',
    'JPN': 'JP',
    'IND': 'IN',
    'LAM': 'RLA',
    'MEA': 'RME',
    'NEU': 'RER',
    'OAS': 'RAS',
    'REF': 'RU',
    'SSA': 'RAF',
    'USA': 'US',
    'APAC': 'RAS',
    'World': 'GLO',
    'RSAM': 'RLA',
    'RCAM': 'RLA',
    'INDO': 'ID',
    'RSAF': 'RAF',
    'CEU': 'RER',
    'SAF': 'ZA',
    'INDIA': 'IN',
    'BRA': 'BR',
    'STAN': 'RAS',
    'WAF': 'RAF',
    'CHN': 'CN',
    'NAF': 'RAF',
    'UKR': 'UA',
    'RSAS': 'RAS',
    'RUS': 'RU',
    'SEAS': 'RAS',
    'KOR': 'KR',
    'JAP': 'JP',
    'EAF': 'RAF',
    'TUR': 'TR',
    'CAN': 'CA',
    'MEX': 'MX',
    'AFR': 'RAF',
    'AUS': 'AU',
    'CSA': 'RLA',
    'CHI': 'CN',
    'EEU': 'RER',
    'FSU': 'RAS',
    'ODA': 'RAS',
    'SKO': 'KR',
    'UK': 'GB',
    'OCE': 'AU',
}

In [39]:
ei_locations.extend(ei_premise_iam_locations_geo_mapping.keys())

In [40]:
locations_spat_biosphere = list(set([i['name'].split(', ')[-1] for i in biosphere_db_spatialized.db_as_list]))

In [41]:
missing_locations_spatialized_biosphere = [location for location in locations_spat_biosphere if (location not in ei_locations) & (location not in ['Deep', 'agricultural', 'in ground', 'mosaic (agroforestry)', 'ocean', 'sole'])]

In [42]:
ei_locations.extend(missing_locations_spatialized_biosphere)

In [43]:
water_loc_list = []
land_loc_list = []
acid_loc_list = []
eutro_loc_list = []
ei_locations = sorted(ei_locations)

ef_cfs_water = ef_cfs[ef_cfs.LCIAMethod_name == 'Water use']
ef_cfs_land = ef_cfs[ef_cfs.LCIAMethod_name == 'Land use']
ef_cfs_acid = ef_cfs[ef_cfs.LCIAMethod_name == 'Acidification']
ef_cfs_eutro = ef_cfs[ef_cfs.LCIAMethod_name == 'Eutrophication, terrestrial']

for loc in ei_locations:
    
    if '-' in loc:
        loc = loc.split('-')[0]
    if 'Canada' in loc:
        loc = 'CA'
        
    if loc in ei_premise_iam_locations_geo_mapping.keys():
        loc = ei_premise_iam_locations_geo_mapping[loc]
    
    if loc in ef_cfs_water.LCIAMethod_location.unique():
        water_loc_list.append(loc)
    else:
        water_loc_list.append('GLO')
    
    if loc in ef_cfs_land.LCIAMethod_location.unique():
        land_loc_list.append(loc)
    else:
        land_loc_list.append('GLO')
    
    if loc in ef_cfs_acid.LCIAMethod_location.unique():
        acid_loc_list.append(loc)
    else:
        acid_loc_list.append('GLO')
    
    if loc in ef_cfs_eutro.LCIAMethod_location.unique():
        eutro_loc_list.append(loc)
    else:
        eutro_loc_list.append('GLO')

In [44]:
df_mapping_ef_ei_geographies = pd.DataFrame(data = [ei_locations, water_loc_list, land_loc_list, acid_loc_list, eutro_loc_list], index = ['ecoinvent', 'water', 'land', 'acid', 'eutro']).T

In [45]:
if write_method_bw:
    df_mapping_ef_ei_geographies.to_excel('results/ei_ef_geo_mapping.xlsx', index=False)